# Inequalities in DIVAnd: Proof of concept

In [ ]:
using DIVAnd
using Makie, CairoMakie, GeoMakie
using LinearAlgebra
using SparseArrays
include("../config.jl")

## Classical analysis

In [ ]:
# observations
x = rand(750);
y = rand(750);
testsize=300

f = (sin.(x*6) .* cos.(y*6).+cos.(x*6))
#f = (sin.(x*6) .* cos.(y*6).+ cos.(x*6)) .+ randn(750) ;

# final grid
xi,yi= ndgrid(range(0,stop=1,length=testsize),range(0,stop=1,length=testsize));

# reference field
fref = (sin.(xi*6) .* cos.(yi*6).+cos.(xi*6))

mask = trues(size(xi));
pm = ones(size(xi)) / (xi[2,1]-xi[1,1]);
pn = ones(size(xi)) / (yi[1,2]-yi[1,1]);

# correlation length
len = 0.4;
# obs. error variance normalized by the background error variance
epsilon2 = 1.;

# fi is the interpolated field
@time fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(x,y),f,len,epsilon2);
@show extrema(fi)

In [ ]:
fig = Figure(size=(900, 400))
ga = Axis(fig[1, 1]; title = "Analytical field and obs.")
sc1 = heatmap!(ga, xi[:,1], yi[1,:], fref, colormap=Reverse(:RdBu))
scatter!(ga, x, y, color=:black, markersize=3)
Colorbar(fig[1, 2], sc1)

ga2 = Axis(fig[1, 3]; title = "Interpolated field")
sc2 = heatmap!(ga2, xi[:,1], yi[1,:], fi, colormap=Reverse(:RdBu))
Colorbar(fig[1, 4], sc2)
fig

## Analysis with positivness
A quick example without any real meaning since imposing a positive solution with a lot of 
negative input values does not make a lot of sense. 

In [ ]:
m=sum(mask)
m2c=DIVAnd.DIVAnd_ineqconstrain(0*ones(Float64,m),Diagonal(ones(Float64,m)))
fi,s=DIVAndrun(mask,(pm,pn),(xi,yi),(x,y),f,len,epsilon2;ineqconstraints=(m2c,));
@show extrema(fi);    

Show where you still violate the inequality. Unless an analysis without inequality constraint satisfies 
the inequality constraint, there will always be points where you violate (slightly) the inequality constraint. 

In [ ]:
fih=deepcopy(fi)
fih[fi.<0.0] .= -1.0;

In [ ]:
fig = Figure(size=(450, 400))
ga = Axis(fig[1, 1]; title = "Analytical field and obs.")
sc1 = heatmap!(ga, xi[:,1], yi[1,:], fih, colormap=Reverse(:RdBu), colorrange=(-1,1))
Colorbar(fig[1, 2], sc1)
fig

## Analysis with inequality (between two values)

In [ ]:
m=sum(mask)
m2ca=DIVAnd.DIVAnd_ineqconstrain(-0.5*ones(Float64,m),Diagonal(ones(Float64,m)))
m2cb=DIVAnd.DIVAnd_ineqconstrain(-0.5*ones(Float64,m),-Diagonal(ones(Float64,m)))
fi,s=DIVAndrun(mask,(pm,pn),(xi,yi),(x,y),f,len,epsilon2;ineqconstraints=(m2ca,m2cb));
@show extrema(fi);

In [ ]:
fig = Figure(size=(450, 400))
ga = Axis(fig[1, 1]; title = "Interpolated field with an inequality constrain")
sc1 = heatmap!(ga, xi[:,1], yi[1,:], fi, colormap=Reverse(:RdBu), colorrange=(-1,1))
Colorbar(fig[1, 2], sc1)
fig

## Increasing in a given direction with gradient of at least 0.1

If you want to try the next cell with variations:

* `for i = 1:1` Impose increasing values in x direction
* `for i = 2:2` Impose increasing values in y direction
* `for i = 1:2` Impose increasing value in oblique direction 

(increase values in both directions is also possible by making two constraints)

In [ ]:
# Creation of gradient operator matrix in direction i
A = spzeros(s.sv.n, s.sv.n)

for i = 1:2
    S = sparse_stagger(size(mask), i, s.iscyclic[i])
    m = (S * mask[:]) .== 1
    A = A +  sparse_pack(mask) * S' * sparse_pack(m)' * s.Dx[i]
end
yo=statevector_pack(s.sv, (0.0 .*fi.+0.1,))
m2c=DIVAnd.DIVAnd_ineqconstrain(yo, A)
fi,s=DIVAndrun(mask,(pm,pn),(xi,yi),(x,y),f,len,epsilon2;ineqconstraints=(m2c,),ntriesmax=20);

In [ ]:
fig = Figure(size=(450, 400))
ga = Axis(fig[1, 1]; title = "Interpolated field")
sc1 = heatmap!(ga, xi[:,1], yi[1,:], fi, colormap=Reverse(:RdBu), colorrange=(-1,1))
Colorbar(fig[1, 2], sc1)
fig

In [ ]:
fig = Figure(size=(950, 400))
ax1 = Axis(fig[1,1])
lines!(ax1, xi[:,10],fi[:,10])
ax2 = Axis(fig[1,2])
lines!(ax2, yi[10,:],fi[10,:])
ax3 = Axis(fig[1,3])
fd=[fi[i,i] for i = 1:size(fi)[1]]
lines!(ax3, fd)
fig

## Other possible inequalities

* Increasing value along trajectories (if closed trajectoy should lead to constant value along closed line)
* Average over certain region larger than a value
* Stable stratification (do we keep the interface for multivariate variables as in `DIVAnd_multivarEOF` ? What about defining which direction is vertical ? Paramater `vertical=` )
* Limitations on gradients (upper and lower values)

<div class="alert alert-block alert-warning">
<h2>⚠️ To be aware of</h2> 
Be sure to think about the background field effect when defining the constraints. <br>
More general question on how to deal with background fields?<br>
Separate creation and tools (user defined or adapted functions) to merge background and anomalies analysis?
</div>